<a href="https://colab.research.google.com/github/Gaurav05082002/collab-notebooks/blob/main/searchmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import pickle

# Load ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False)

# Function to preprocess and get feature maps
def get_feature_map(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature_map = base_model.predict(x)
    return feature_map.flatten()

# Load the CSV file
csv_file_path = '/content/all category.csv'
df = pd.read_csv(csv_file_path)

# Process each image and get its feature map
feature_maps = []

for index, row in df.iterrows():
    img_url = row['imgurl']
    try:
        feature_map = get_feature_map(img_url)
        feature_maps.append(feature_map)
    except Exception as e:
        print(f"Error processing image at {img_url}: {e}")
        feature_maps.append(None)  # Append None if there's an error

# Save the feature maps and links to a file
data = {
    'links': df['linkn'].tolist(),
    'feature_maps': feature_maps
}

with open('feature_maps.pkl', 'wb') as f:
    pickle.dump(data, f)

print("Feature maps have been saved to feature_maps.pkl")


1/1 [==============================] - 0s 161ms/step
Error processing image at https://m.media-amazon.com/images/I/61NAaXkZWlL.__AC_SX300_SY300_QL70_ML2_.jpg: index 1 is out of bounds for axis 3 with size 1
1/1 [==============================] - 0s 165ms/step
Error processing image at https://m.media-amazon.com/images/I/61LxO7N5UhL.__AC_SX300_SY300_QL70_ML2_.jpg: index 1 is out of bounds for axis 3 with size 1
1/1 [==============================] - 0s 246ms/step
Error processing image at https://m.media-amazon.com/images/I/614l3Z3zxjL.__AC_SX342_SY445_QL70_ML2_.jpg: index 1 is out of bounds for axis 3 with size 1
1/1 [==============================] - 0s 295ms/step
Error processing image at https://m.media-amazon.com/images/I/71CrxRaaiwL.__AC_SX300_SY300_QL70_ML2_.jpg: index 1 is out of bounds for axis 3 with size 1
1/1 [==============================] - 0s 166ms/step
Feature maps have been saved to feature_maps.pkl


In [11]:
pip install scikit-learn


In [15]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pickle

# Load the feature maps and links from the saved file
with open('feature_maps.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract feature maps and corresponding links, excluding any None entries
feature_maps = np.array([f for f in data['feature_maps'] if f is not None])
links = [data['links'][i] for i, f in enumerate(data['feature_maps']) if f is not None]

# Check if we have feature maps
if len(feature_maps) == 0:
    raise ValueError("No feature maps to train on. Please check your data.")

# Train a NearestNeighbors model
knn = NearestNeighbors(n_neighbors=1, metric='cosine')
knn.fit(feature_maps)

# Save the trained KNN model
with open('knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

print("KNN model has been saved to knn_model.pkl")


KNN model has been saved to knn_model.pkl


In [17]:
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import pickle
import os

# Function to preprocess and get feature maps
def get_feature_map(image_path):
    if os.path.exists(image_path):  # Check if the path is a local file path
        img = Image.open(image_path).resize((224, 224))
    else:  # Otherwise, assume it's a URL
        response = requests.get(image_path)
        img = Image.open(BytesIO(response.content)).resize((224, 224))

    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature_map = base_model.predict(x)
    return feature_map.flatten()

# Load the KNN model and data
with open('knn_model.pkl', 'rb') as f:
    knn = pickle.load(f)

with open('feature_maps.pkl', 'rb') as f:
    data = pickle.load(f)

links = [data['links'][i] for i, f in enumerate(data['feature_maps']) if f is not None]

# Load ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False)

# Function to find the most similar product link
def find_similar_product(image_path):
    feature_map = get_feature_map(image_path)
    distances, indices = knn.kneighbors([feature_map])
    return links[indices[0][0]]

# Example usage
test_img_path = '/content/watchpic.webp'  # Local file path
similar_product_link = find_similar_product(test_img_path)
print(f"The most similar product link: {similar_product_link}")

# test_img_url = 'http://example.com/path/to/your/image.jpg'  # URL example
# similar_product_link = find_similar_product(test_img_url)
# print(f"The most similar product link: {similar_product_link}")


1/1 [==============================] - 1s 1s/step
The most similar product link: https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyMzg3ODg5NzMxNjY4MjQ6MTcxODYxNDkwMzpzcF9zZWFyY2hfdGhlbWF0aWM6OjozOkIwMTVaWFBVTjg6c2VhcmNo&url=%2FCasio-LTP-V005GL-LTP-V007L-9EUDF-Wristwatch%2Fdp%2FB015ZXPUN8%2Fref%3Dsxin_29_pa_sp_search_thematic_sspa%3Fcontent-id%3Damzn1.sym.1eb3ca68-a7a7-4620-b6d3-e5aa25dca968%253Aamzn1.sym.1eb3ca68-a7a7-4620-b6d3-e5aa25dca968%26cv_ct_cx%3Dwatch%26dib%3DeyJ2IjoiMSJ9.ZI-z43IfbxYNM3CYCpY6rWJJQD0SwiN4KzCnr6h1k39wa0JrWJ9PaISi1XrcICfklItTFmoNzi3SL-0B09_DXum2QIYtGmAgYhrbHfRi1pOZ-XK2S3_ROtBQb4uDoKW-.EmPIf9VqOQKbVdCVjh2jKLTh7ccTGXPbyzXsdPbCrpE%26dib_tag%3Dse%26keywords%3Dwatch%26pd_rd_i%3DB015ZXPUN8%26pd_rd_r%3Dfc4d3e51-4c8b-468d-a453-7ca0d9042f48%26pd_rd_w%3DX1ibB%26pd_rd_wg%3D7SsKL%26pf_rd_p%3D1eb3ca68-a7a7-4620-b6d3-e5aa25dca968%26pf_rd_r%3D28E7S82P32PPB8N1RS1F%26qid%3D1718614903%26sbo%3DRZvfv%252F%252FHxDF%252BO5021pAnSA%253D%253D%26sr%3D1-4-6b0a36a0-7223-4d6d-a074-78a833921c